In [1]:
import gym
import torch
import numpy as np
import sys
from torchvision.transforms import ToTensor, Lambda
from torch import nn
from my_model import DQN
from utils import dataset, dataloader, buffer
import trainer
import random
import torch.onnx as onnx
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('RLresult/hopper/dqn')
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [2]:
env = gym.make('Hopper-v2') 
print(env.action_space)
print(env.observation_space)

Box([-1. -1. -1.], [1. 1. 1.], (3,), float32)
Box([-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf inf inf inf inf inf], (11,), float64)


In [3]:
HIDDEN_SIZE = 32
BATCH_SIZE = 10000
learning_rate = 0.01
HOP_ACTION = 125
CAPACITY = 10000
TRAIN_ITER = CAPACITY*10/BATCH_SIZE
PARAM_PATH = 'DQN_param/hopperDQN.pth'
PARAM_PATH_TEST = 'DQN_param/hopperDQN_test.pth'
#print(Transition(2,1,2,3))
#Transition(state=2, action=1, next_state=2, reward=3)



In [16]:
%%time

myhopdata = datset.EnvData(capacity = CAPACITY)

#dqn_dataloader = DataLoader(myhopdata, batch_size=64, shuffle=True)

input_state_size = len(env.observation_space.sample())

updatedDQN = DQN.DQN(input_state_size, HIDDEN_SIZE, CART_ACTION).to(device)
baseDQN = DQN.DQN(input_state_size, HIDDEN_SIZE, CART_ACTION).to(device)
mybuffer = buffer.ForwardMem(baseDQN, env, "hope")


updatedDQN.load_state_dict(torch.load(PARAM_PATH))


baseDQN.load_state_dict(updatedDQN.state_dict())
baseDQN.eval()

dqn_dataloader = dataloader.CustomDataLoader(myhopdata, batch_size=BATCH_SIZE)
#len(env.observation_space.sample())
optimizer = torch.optim.SGD(updatedDQN.parameters(), lr=learning_rate)
mytrain = trainer.Train("hope", dqn_dataloader, updatedDQN, baseDQN)

train_iter = 100
i = 0
#for p in updatedDQN.parameters(): 
#    print(p)

while(i < train_iter):
    i = i + 1
    renewal_memory(CAPACITY, rendering = False)
    loss = mytrain.training(TRAIN_ITER, BATCH_SIZE, optimizer)
    writer.add_scalar("loss", loss, i)
    torch.save(updatedDQN.state_dict(), PARAM_PATH)
    baseDQN.load_state_dict(updatedDQN.state_dict())
    baseDQN.eval()

env.close()


/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


loss =  tensor(1165896.3750, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(178925.9375, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(32147.1152, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(24535.7793, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(24160.0898, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(24238.2773, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(24101.2734, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(24090.2559, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(24030.8535, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(24724.3574, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(23868.5820, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(23921.8750, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(23947.4082, device='cuda:0', grad_fn=<MseLossBackward>)


KeyboardInterrupt: 

In [10]:
#writer.add_graph(updatedDQN, myhopdata[0][0])
#writer.add_graph(baseDQN, myhopdata[0][0])


In [11]:
writer.flush()
writer.close()

In [12]:
'''
print(env.unwrapped.sim.model.get_joint_qpos_addr('root'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('hip_1'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('hip_2'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('hip_3'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('hip_4'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_1'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_2'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_3'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_4'))
#if u enter <body> in addr then this will show you available list
#('root', 'hip_1', 'ankle_1', 'hip_2', 'ankle_2', 'hip_3', 'ankle_3', 'hip_4', 'ankle_4')
#qpos is front value index
'''


"\nprint(env.unwrapped.sim.model.get_joint_qpos_addr('root'))\nprint(env.unwrapped.sim.model.get_joint_qpos_addr('hip_1'))\nprint(env.unwrapped.sim.model.get_joint_qpos_addr('hip_2'))\nprint(env.unwrapped.sim.model.get_joint_qpos_addr('hip_3'))\nprint(env.unwrapped.sim.model.get_joint_qpos_addr('hip_4'))\nprint(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_1'))\nprint(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_2'))\nprint(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_3'))\nprint(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_4'))\n#if u enter <body> in addr then this will show you available list\n#('root', 'hip_1', 'ankle_1', 'hip_2', 'ankle_2', 'hip_3', 'ankle_3', 'hip_4', 'ankle_4')\n#qpos is front value index\n"

In [13]:
target = torch.empty(3, dtype=torch.long).random_(5)

In [14]:
target

tensor([1, 4, 1])

In [15]:
input = torch.randn(3, 5, requires_grad=True)